## Data Clean
[Data Source: U.S. Department of Health & Human Services](https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh)

In [1]:
# Import packages
import pandas as pd

In [26]:
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import dataset
df = pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries__RAW__20231105.csv')
df.head()

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,previous_day_admission_pediatric_covid_confirmed_5_11,previous_day_admission_pediatric_covid_confirmed_5_11_coverage,previous_day_admission_pediatric_covid_confirmed_unknown,previous_day_admission_pediatric_covid_confirmed_unknown_coverage,staffed_icu_pediatric_patients_confirmed_covid,staffed_icu_pediatric_patients_confirmed_covid_coverage,staffed_pediatric_icu_bed_occupancy,staffed_pediatric_icu_bed_occupancy_coverage,total_staffed_pediatric_icu_beds,total_staffed_pediatric_icu_beds_coverage
0,IA,2021/05/01,0,44,82,2,41,83,0.0,124,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1,WY,2021/05/01,3,25,3,4,24,3,0.0,29,...,NaN,0,NaN,0,NaN,0,0.0,8,0.0,8
2,ND,2021/04/30,12,38,1,11,39,1,27.0,50,...,NaN,0,NaN,0,NaN,0,46.0,7,74.0,7
3,ID,2021/04/29,4,40,8,4,40,8,4.0,51,...,NaN,0,NaN,0,NaN,0,79.0,30,229.0,30
4,UT,2021/04/29,1,57,1,1,57,1,2.0,58,...,NaN,0,NaN,0,NaN,0,253.0,58,410.0,58


In [3]:
# Create time-series variables
df['date'] = pd.to_datetime(df['date'])
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Day'] = df['date'].dt.day

#### 1st dataframe -- COVID Cases
Creating this dataframe as the COVID Cases. Detailedly, we select variables:

- state: The two digit state code.

- date (Month, Year, Day): The date of the report.

- previous_day_admission_covid_confirmed: Number of patients who were admitted to an inpatient bed on the previous calendar day who had confirmed COVID-19 at the time of admission in this state.

- previous_day_admission_covid_confirmed_coverage: Number of hospitals reporting "previous_day_admission_covid_confirmed" in this state.

- previous_day_admission_covid_suspected: Number of patients who were admitted to an inpatient bed on the previous calendar day who had suspected COVID-19 at the time of admission in this state.

- previous_day_admission_covid_suspected_coverage: Number of hospitals reporting "previous_day_admission_covid_suspected" in this state.

In [5]:
df1 = df[['state', 'Month', 'Year', 'Day','previous_day_admission_adult_covid_confirmed',
       'previous_day_admission_adult_covid_confirmed_coverage',
       'previous_day_admission_adult_covid_suspected',
       'previous_day_admission_adult_covid_suspected_coverage',
       'previous_day_admission_pediatric_covid_confirmed',
       'previous_day_admission_pediatric_covid_confirmed_coverage',
       'previous_day_admission_pediatric_covid_suspected',
       'previous_day_admission_pediatric_covid_suspected_coverage']]
df1.head()

,state,Month,Year,Day,previous_day_admission_adult_covid_confirmed,previous_day_admission_adult_covid_confirmed_coverage,previous_day_admission_adult_covid_suspected,previous_day_admission_adult_covid_suspected_coverage,previous_day_admission_pediatric_covid_confirmed,previous_day_admission_pediatric_covid_confirmed_coverage,previous_day_admission_pediatric_covid_suspected,previous_day_admission_pediatric_covid_suspected_coverage
0,IA,5,2021,1,26.0,126,41.0,124,0.0,124,3.0,124
1,WY,5,2021,1,3.0,31,5.0,29,0.0,29,0.0,29
2,ND,4,2021,30,12.0,51,11.0,50,0.0,50,0.0,50
3,ID,4,2021,29,12.0,52,5.0,51,1.0,51,1.0,51
4,UT,4,2021,29,18.0,59,1.0,58,1.0,58,0.0,58


In [6]:
# Specify columns to keep (state, Month, Year, Day) and columns to melt
columns_to_keep = ['state', 'Month', 'Year', 'Day']
columns_to_melt = df1.columns.difference(columns_to_keep)

# Melt the DataFrame
df_long = pd.melt(df1, id_vars=columns_to_keep, value_vars=columns_to_melt, var_name='variable')

# Split the variable column into coverage, number, and age group
df_long['group'] = [i.split('_')[3] for i in df_long.variable]
df_long['covid'] = [i.split('_')[5] for i in df_long.variable]
df_long['coverage'] = ['hospital' if i.split('_')[-1] == 'coverage' else 'patients'
                  for i in df_long.variable]

In [7]:
df_month1 = df_long.groupby(['state','Year','Month','group','covid','coverage']).value.sum().reset_index()
df_month1.head()

,state,Year,Month,group,covid,coverage,value
0,AK,2020,3,adult,confirmed,hospital,0.0
1,AK,2020,3,adult,confirmed,patients,0.0
2,AK,2020,3,adult,suspected,hospital,0.0
3,AK,2020,3,adult,suspected,patients,0.0
4,AK,2020,3,pediatric,confirmed,hospital,0.0


#### 2nd Dataframe -- COVID Cases by Age Band
Creating this dataframe as the COVID Cases by Age Band. Detailedly, we select variables:

- state: The two digit state code.

- date (Month, Year, Day): The date of the report.

- previous_day_admission_covid_confirmed: Number of patients who were admitted to an inpatient bed on the previous calendar day who had confirmed COVID-19 at the time of admission in this state.

- previous_day_admission_covid_confirmed_coverage: Number of hospitals reporting "previous_day_admission_covid_confirmed" in this state.

- previous_day_admission_covid_suspected: Number of patients who were admitted to an inpatient bed on the previous calendar day who had suspected COVID-19 at the time of admission in this state.

- previous_day_admission_covid_suspected_coverage: Number of hospitals reporting "previous_day_admission_covid_suspected" in this state.

In [29]:
# Select the variables
lis1 = df.columns[-77:-42].to_list()
lis1

['previous_day_admission_adult_covid_confirmed_18-19',
 'previous_day_admission_adult_covid_confirmed_18-19_coverage',
 'previous_day_admission_adult_covid_confirmed_20-29',
 'previous_day_admission_adult_covid_confirmed_20-29_coverage',
 'previous_day_admission_adult_covid_confirmed_30-39',
 'previous_day_admission_adult_covid_confirmed_30-39_coverage',
 'previous_day_admission_adult_covid_confirmed_40-49',
 'previous_day_admission_adult_covid_confirmed_40-49_coverage',
 'previous_day_admission_adult_covid_confirmed_50-59',
 'previous_day_admission_adult_covid_confirmed_50-59_coverage',
 'previous_day_admission_adult_covid_confirmed_60-69',
 'previous_day_admission_adult_covid_confirmed_60-69_coverage',
 'previous_day_admission_adult_covid_confirmed_70-79',
 'previous_day_admission_adult_covid_confirmed_70-79_coverage',
 'previous_day_admission_adult_covid_confirmed_80+',
 'previous_day_admission_adult_covid_confirmed_80+_coverage',
 'previous_day_admission_adult_covid_confirmed_unkno

In [28]:
lis2 = ['previous_day_admission_pediatric_covid_confirmed_0_4',
 'previous_day_admission_pediatric_covid_confirmed_0_4_coverage',
 'previous_day_admission_pediatric_covid_confirmed_12_17',
 'previous_day_admission_pediatric_covid_confirmed_12_17_coverage',
 'previous_day_admission_pediatric_covid_confirmed_5_11',
 'previous_day_admission_pediatric_covid_confirmed_5_11_coverage',
 'previous_day_admission_pediatric_covid_confirmed_unknown',
 'previous_day_admission_pediatric_covid_confirmed_unknown_coverage']

In [17]:
df2 = df[['state','Month','Year','Day'] + lis1+lis2]
df2.head()

,state,Month,Year,Day,previous_day_admission_adult_covid_confirmed_18-19,previous_day_admission_adult_covid_confirmed_18-19_coverage,previous_day_admission_adult_covid_confirmed_20-29,previous_day_admission_adult_covid_confirmed_20-29_coverage,previous_day_admission_adult_covid_confirmed_30-39,previous_day_admission_adult_covid_confirmed_30-39_coverage,...,previous_day_admission_adult_covid_suspected_80+_coverage,previous_day_admission_adult_covid_suspected_unknown,previous_day_admission_pediatric_covid_confirmed_0_4,previous_day_admission_pediatric_covid_confirmed_0_4_coverage,previous_day_admission_pediatric_covid_confirmed_12_17,previous_day_admission_pediatric_covid_confirmed_12_17_coverage,previous_day_admission_pediatric_covid_confirmed_5_11,previous_day_admission_pediatric_covid_confirmed_5_11_coverage,previous_day_admission_pediatric_covid_confirmed_unknown,previous_day_admission_pediatric_covid_confirmed_unknown_coverage
0,IA,5,2021,1,0.0,126,4.0,126,3.0,126,...,124,0.0,NaN,0,NaN,0,NaN,0,NaN,0
1,WY,5,2021,1,0.0,31,0.0,31,0.0,31,...,29,0.0,NaN,0,NaN,0,NaN,0,NaN,0
2,ND,4,2021,30,0.0,51,0.0,51,0.0,51,...,50,0.0,NaN,0,NaN,0,NaN,0,NaN,0
3,ID,4,2021,29,0.0,52,0.0,52,0.0,52,...,51,0.0,NaN,0,NaN,0,NaN,0,NaN,0
4,UT,4,2021,29,0.0,59,2.0,59,3.0,59,...,58,0.0,NaN,0,NaN,0,NaN,0,NaN,0


In [18]:
# Specify columns to keep (state, Month, Year, Day) and columns to melt
columns_to_keep = ['state', 'Month', 'Year', 'Day']
columns_to_melt = df2.columns.difference(columns_to_keep)

# Melt the DataFrame
df_long = pd.melt(df2, id_vars=columns_to_keep, value_vars=columns_to_melt, var_name='variable')

# Split the variable column into coverage, number, and age group
df_long['group'] = [i.split('_')[3] for i in df_long.variable]
df_long['covid'] = [i.split('_')[5] for i in df_long.variable]
df_long['age'] = [i.split('_')[-1] if i.split('_')[-1] != 'coverage' else i.split('_')[-2]
                  for i in df_long.variable]
df_long['coverage'] = ['hospital' if i.split('_')[-1] == 'coverage' else 'patients'
                  for i in df_long.variable]

In [19]:
df_month = df_long.groupby(['state','Year','Month','group','covid','age','coverage']).value.sum().reset_index()
df_month['Age'] = ['0-4' if i == '4' else
                  '5-11' if i == '11' else
                  '12-17' if i == '17' else
                  str(i) for i in df_month['age']]
df_month.head()

,state,Year,Month,group,covid,age,coverage,value,Age
0,AK,2020,3,adult,confirmed,18-19,hospital,0.0,18-19
1,AK,2020,3,adult,confirmed,18-19,patients,0.0,18-19
2,AK,2020,3,adult,confirmed,20-29,hospital,0.0,20-29
3,AK,2020,3,adult,confirmed,20-29,patients,0.0,20-29
4,AK,2020,3,adult,confirmed,30-39,hospital,0.0,30-39


#### 3rd Dataframe -- Inpatient Bed Number
Creating this dataframe as the Inpatient Bed Number. 

Detailedly, we select variables:

- state: The two digit state code.

- date (Month, Year, Day): The date of the report.

- inpatient_beds: Reported total number of staffed inpatient beds including all overflow and surge/expansion beds used for inpatients (includes all ICU beds) in this state.

- inpatient_beds_coverage: Number of hospitals reporting "inpatient_beds" in this state.

- inpatient_beds_used: Reported total number of staffed inpatient beds that are occupied in this state.

- inpatient_beds_used_coverage: Number of hospitals reporting "inpatient_beds_used" in this state.

- inpatient_beds_used_covid: Reported patients currently hospitalized in an inpatient bed who have suspected or confirmed COVID-19 in this state.

- inpatient_beds_used_covid_coverage: Number of hospitals reporting "inpatient_beds_used_covid" in this state.

In [21]:
df3 = df[['state','Month','Year','Day','inpatient_beds', 'inpatient_beds_coverage', 'inpatient_beds_used',
       'inpatient_beds_used_coverage', 'inpatient_beds_used_covid',
       'inpatient_beds_used_covid_coverage']]
df3.head()

,state,Month,Year,Day,inpatient_beds,inpatient_beds_coverage,inpatient_beds_used,inpatient_beds_used_coverage,inpatient_beds_used_covid,inpatient_beds_used_covid_coverage
0,IA,5,2021,1,8050.0,126,4833.0,126,273.0,124
1,WY,5,2021,1,1702.0,31,795.0,31,33.0,29
2,ND,4,2021,30,2451.0,51,1802.0,51,120.0,50
3,ID,4,2021,29,3740.0,52,2179.0,52,136.0,51
4,UT,4,2021,29,6212.0,59,3792.0,59,146.0,58


In [22]:
# Specify columns to keep (state, Month, Year, Day) and columns to melt
columns_to_keep = ['state', 'Month', 'Year', 'Day']
columns_to_melt = df3.columns.difference(columns_to_keep)

# Melt the DataFrame
df_long = pd.melt(df3, id_vars=columns_to_keep, value_vars=columns_to_melt, var_name='variable')

# Split the variable column into coverage, number, and age group
df_long['coverage'] = ['hospital' if i.split('_')[-1] == 'coverage' else 'patients'
                  for i in df_long.variable]

df_long['Variable'] = [i[:-9] if '_coverage' in i else i for i in df_long['variable']]

In [23]:
# Group by state, year, month, coverage and variable
df_month2 = df_long.groupby(['state','Year','Month','coverage','Variable']).value.sum().reset_index()
df_month2.head()

,state,Year,Month,coverage,Variable,value
0,AK,2020,3,hospital,inpatient_beds,15.0
1,AK,2020,3,hospital,inpatient_beds_used,15.0
2,AK,2020,3,hospital,inpatient_beds_used_covid,15.0
3,AK,2020,3,patients,inpatient_beds,950.0
4,AK,2020,3,patients,inpatient_beds_used,369.0


#### 4th Dataframe -- Distribution of Death Cases By State
Creating this dataframe as the Distribution of Death Cases By State. 

Detailedly, we select variables:

- state: The two digit state code.

- date (Month, Year, Day): The date of the report.

- deaths_covid: Number of patients with suspected or confirmed COVID-19 who died on the previous calendar day in the hospital, ED, or overflow location

- deaths_covid_coverage: The number of hospital facilities that reported deaths_covid for the given time period.

In [24]:
# Select the variables
dff = df[['state','Month','Year','Day','deaths_covid','deaths_covid_coverage']]
dff.head()

,state,Month,Year,Day,deaths_covid,deaths_covid_coverage
0,IA,5,2021,1,1.0,124
1,WY,5,2021,1,0.0,29
2,ND,4,2021,30,0.0,50
3,ID,4,2021,29,1.0,51
4,UT,4,2021,29,0.0,58


In [27]:
# Group by state, year and month
dfff = dff.groupby(['state','Year','Month'])['deaths_covid','deaths_covid_coverage'].sum().reset_index()
dfff.head()

,state,Year,Month,deaths_covid,deaths_covid_coverage
0,AK,2020,3,0.0,15
1,AK,2020,4,4.0,709
2,AK,2020,5,15.0,641
3,AK,2020,6,5.0,570
4,AK,2020,7,12.0,686
